Actividad guiada 3
Salvador Díaz González
GitHub:

Google Colab: https://colab.research.google.com/drive/1crY0E_vVS8mGwJA7i7tudzJ_A3NkSAZI?usp=sharing

In [ ]:
!pip install requests  #Hacer llamadas http a paginas de la red
!pip install tsplib95  #Modulo para las instancias del problema del TSP

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 23.2 MB/s eta 0:00:00
  Attempting uninstall: networkx
    Found existing installation: networkx 3.1
    Uninstalling networkx-3.1:
      Successfully uninstalled networkx-3.1


In [ ]:
import urllib.request #Hacer llamadas http a paginas de la red
import tsplib95       #Modulo para las instancias del problema del TSP
import math           #Modulo de funciones matematicas. Se usa para exp
import random         #Para generar valores aleatorios

#Descargamos el fichero de datos(Matriz de distancias)
file = "swiss42.tsp" ;
urllib.request.urlretrieve("http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/tsp/swiss42.tsp.gz", file + '.gz')
!gzip -d swiss42.tsp.gz     #Descomprimir el fichero de datos

In [ ]:
#Carga de datos y generación de objeto problem
###############################################################################
problem = tsplib95.load(file)

#Nodos
Nodos = list(problem.get_nodes())

#Aristas
Aristas = list(problem.get_edges())


In [ ]:
#Se genera una solucion aleatoria con comienzo en en el nodo 0
def crear_solucion(Nodos):
  solucion = [Nodos[0]]
  for n in Nodos[1:]:
    solucion = solucion + [random.choice(list(set(Nodos) - set(solucion)))]
  return solucion

crear_solucion(Nodos)

#Devuelve la distancia entre dos nodos
def distancia(a,b, problem):
  return problem.get_weight(a,b)

#Devuelve la distancia total de una trayectoria/solucion
def distancia_total(solucion, problem):
  distancia_total = 0
  for i in range(len(solucion)-1):
    distancia_total += distancia(solucion[i] ,solucion[i+1] ,  problem)
  return distancia_total + distancia(solucion[len(solucion)-1] ,solucion[0], problem)






In [ ]:
def busqueda_aleatoria(problem, N):
  #N es el numero de iteraciones
  Nodos = list(problem.get_nodes())

  mejor_solucion = []
  #mejor_distancia = 10e100                         #Inicializamos con un valor alto
  mejor_distancia = float('inf')                    #Inicializamos con un valor alto

  for i in range(N):                                #Criterio de parada: repetir N veces pero podemos incluir otros
    solucion = crear_solucion(Nodos)                #Genera una solucion aleatoria
    distancia = distancia_total(solucion, problem)  #Calcula el valor objetivo(distancia total)

    if distancia < mejor_distancia:                 #Compara con la mejor obtenida hasta ahora
      mejor_solucion = solucion
      mejor_distancia = distancia


  print("Mejor solución:" , mejor_solucion)
  print("Distancia     :" , mejor_distancia)
  return mejor_solucion



#Busqueda aleatoria con 5000 iteraciones
solucion = busqueda_aleatoria(problem, 5000)

Mejor solución: [0, 27, 1, 2, 31, 7, 13, 19, 41, 8, 25, 26, 12, 29, 16, 14, 10, 23, 39, 21, 24, 18, 20, 17, 33, 37, 35, 36, 38, 11, 30, 28, 22, 34, 40, 3, 4, 15, 32, 9, 6, 5]
Distancia     : 3674


In [ ]:
solucion = busqueda_aleatoria(problem, 5000)

def genera_vecina(solucion):
  #Generador de soluciones vecinas: 2-opt (intercambiar 2 nodos) Si hay N nodos se generan (N-1)x(N-2)/2 soluciones
  #Se puede modificar para aplicar otros generadores distintos que 2-opt
  #print(solucion)
  mejor_solucion = []
  mejor_distancia = 10e100
  for i in range(1,len(solucion)-1):          #Recorremos todos los nodos en bucle doble para evaluar todos los intercambios 2-opt
    for j in range(i+1, len(solucion)):

      #Se genera una nueva solución intercambiando los dos nodos i,j:
      #  (usamos el operador + que para listas en python las concatena) : ej.: [1,2] + [3] = [1,2,3]
      vecina = solucion[:i] + [solucion[j]] + solucion[i+1:j] + [solucion[i]] + solucion[j+1:]

      #Se evalua la nueva solución ...
      distancia_vecina = distancia_total(vecina, problem)

      #... para guardarla si mejora las anteriores
      if distancia_vecina <= mejor_distancia:
        mejor_distancia = distancia_vecina
        mejor_solucion = vecina
  return mejor_solucion

print("Distancia Solucion Incial:" , distancia_total(solucion, problem))

nueva_solucion = genera_vecina(solucion)
print("Distancia Mejor Solucion Local:", distancia_total(nueva_solucion, problem))


Mejor solución: [0, 20, 35, 40, 12, 23, 34, 28, 26, 33, 31, 17, 4, 8, 2, 30, 36, 15, 29, 22, 39, 9, 21, 11, 24, 25, 10, 41, 13, 1, 6, 3, 7, 38, 32, 5, 27, 18, 19, 16, 37, 14]
Distancia     : 3716
Distancia Solucion Incial: 3716
Distancia Mejor Solucion Local: 3282


In [ ]:
def busqueda_local(problem):
  mejor_solucion = []

  #Generar una solucion inicial de referencia(aleatoria)
  solucion_referencia = crear_solucion(Nodos)
  mejor_distancia = distancia_total(solucion_referencia, problem)
  print(mejor_distancia)
  iteracion=0             #Un contador para saber las iteraciones que hacemos
  while(1):
    iteracion +=1         #Incrementamos el contador
    #print('#',iteracion)

    #Obtenemos la mejor vecina ...
    vecina = genera_vecina(solucion_referencia)

    #... y la evaluamos para ver si mejoramos respecto a lo encontrado hasta el momento
    distancia_vecina = distancia_total(vecina, problem)

    #Si no mejoramos hay que terminar. Hemos llegado a un minimo local(según nuestro operador de vencindad 2-opt)
    if distancia_vecina < mejor_distancia:
      #mejor_solucion = copy.deepcopy(vecina)   #Con copia profunda. Las copias en python son por referencia
      mejor_solucion = vecina                   #Guarda la mejor solución encontrada
      mejor_distancia = distancia_vecina

    else:
      print("En la iteracion ", iteracion, ", la mejor solución encontrada es:" , mejor_solucion)
      print("Distancia     :" , mejor_distancia)
      return mejor_solucion

    solucion_referencia = vecina


sol = busqueda_local(problem )

5033
En la iteracion  37 , la mejor solución encontrada es: [0, 7, 37, 15, 16, 14, 19, 13, 12, 11, 25, 39, 22, 38, 30, 28, 2, 27, 3, 4, 6, 5, 26, 18, 10, 40, 24, 21, 34, 33, 20, 32, 8, 41, 23, 9, 29, 31, 35, 36, 17, 1]
Distancia     : 1824


Se ha introducido:
 - numero_ejecuciones (el número de ejecuciones de búsqueda local a realizar)
 - max_iteraciones (el número máximo de iteraciones permitidas por ejecución).

In [ ]:
def busqueda_local(problem, ejecuciones, iteraciones):
  mejor_solucion = []
  mejor_distancia = float('inf')

  for _ in range(ejecuciones):  # Número de ejecuciones de búsqueda local
    # Generar una solución inicial de referencia (aleatoria)
    solucion_referencia = crear_solucion(Nodos)
    distancia_referencia = distancia_total(solucion_referencia, problem)

    iteracion = 0  # Un contador para saber las iteraciones que hacemos
    while True:
      iteracion += 1  # Incrementamos el contador

      # Obtenemos la mejor vecina ...
      vecina = genera_vecina(solucion_referencia)

      # ... y la evaluamos para ver si mejoramos respecto a lo encontrado hasta el momento
      distancia_vecina = distancia_total(vecina, problem)

      # Si no mejoramos, hay que terminar. Hemos llegado a un mínimo local (según nuestro operador de vecindad 2-opt)
      if distancia_vecina < distancia_referencia:
        mejor_solucion = vecina  # Guarda la mejor solución encontrada
        mejor_distancia = distancia_vecina
        distancia_referencia = distancia_vecina

      if iteracion >= iteraciones:
        break

      solucion_referencia = vecina

  print("La mejor solución encontrada es:", mejor_solucion)
  print("Distancia:", mejor_distancia)
  return mejor_solucion

ejecuciones = 5  # Número de ejecuciones de búsqueda local
iteraciones = 100  # Número máximo de iteraciones por ejecución

sol = busqueda_local(problem, ejecuciones, iteraciones)


La mejor solución encontrada es: [0, 31, 17, 1, 10, 25, 11, 12, 18, 26, 6, 32, 30, 22, 29, 28, 2, 27, 3, 4, 8, 41, 23, 9, 39, 21, 40, 24, 38, 34, 33, 20, 35, 36, 37, 7, 15, 16, 14, 19, 13, 5]
Distancia: 1685
